In [1]:
import os, sys

from utils.mining_data_tb import OmdbCleaner, NetCleaner, WordCleaner

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jgnsa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jgnsa\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
rootpath = os.path.dirname(os.getcwd())
rootpath

'd:\\Documentos\\TheBridge\\bridge_datascience_JorgeGarcia\\EDA_FINAL'

In [3]:
#PATHS
data_path = rootpath + os.sep + 'data'
net_raw_path = data_path + os.sep + 'Netflix_raw'
net_movies_path = net_raw_path + os.sep + 'netflix_movie_titles.csv'
imdb_path = data_path + os.sep + 'titles.tsv'
omdb_net_path = data_path + os.sep + 'OMDb_Netflix'
netid_path = net_raw_path + os.sep + 'Net_Data_ID.csv'

In [4]:
omdb = OmdbCleaner(net_path=net_movies_path, imdb_path=imdb_path, omdb_net_path=omdb_net_path)

In [5]:
omdb.get_imdb_ids(save=True)

,netflix_id,startYear,originalTitle,titleId,titleType,primaryTitle,isAdult,endYear,runtimeMinutes,genres
0,1,2003,Dinosaur Planet,tt0389605,tvMiniSeries,Dinosaur Planet,0,\N,50,"Animation,Documentary,Family"
1,12,1947,My Favorite Brunette,tt0039645,movie,My Favorite Brunette,0,\N,87,"Comedy,Crime,Mystery"
2,15,1988,Neil Diamond: Greatest Hits Live,tt0168093,video,Neil Diamond: Greatest Hits Live,0,\N,60,"Documentary,Music"
3,17,2005,7 Seconds,tt0417395,video,7 Seconds,0,\N,96,"Action,Comedy,Crime"
4,18,1994,Immortal Beloved,tt0110116,movie,Immortal Beloved,0,\N,121,"Biography,Drama,Music"
...,...,...,...,...,...,...,...,...,...,...
9726,17762,1997,Gattaca,tt0119177,movie,Gattaca,0,\N,106,"Drama,Sci-Fi,Thriller"
9727,17763,1978,Interiors,tt0077742,movie,Interiors,0,\N,92,Drama
9728,17764,1998,Shakespeare in Love,tt0138097,movie,Shakespeare in Love,0,\N,123,"Comedy,Drama,History"
9729,17769,2003,The Company,tt0335013,movie,The Company,0,\N,112,"Drama,Music,Romance"


In [5]:
netid_path = data_path + os.sep + 'Netflix_raw' + os.sep + 'Net_Data_ID.csv'

omdb.omdb_to_csv(netid_path, savepath=omdb_net_path, row_start=10, row_num=10) #Ejemplo de llamada a API

1 Response: True
2 Response: True
3 Response: True
4 Response: True
5 Response: True
6 Response: True
7 Response: True
8 Response: True
9 Response: True
10 Response: True
11 Response: True


,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Type,DVD,BoxOffice,Production,Website,Response,Season,Episode,seriesID,totalSeasons
0,Never Die Alone,2004,R,26 Mar 2004,88 min,"Action, Crime, Drama, Thriller",Ernest R. Dickerson,"Donald Goines (novel), James Gibson (screenplay)","DMX, Michael Ealy, Drew Sidora, Antwon Tanner","A film noir centering around a hard-boiled, st...",...,movie,18 Sep 2012,"$5,645,298","Fearon Entertainment, Visionbox Pictures, Cont...",N/A,True,NaN,NaN,NaN,NaN
1,Lilo and Stitch,2002,N/A,15 Mar 2002,N/A,Talk-Show,Art Spigel,N/A,"Mike Johnson, Alyson Stoner",N/A,...,episode,NaN,NaN,NaN,NaN,True,1,9,tt0382488,NaN
2,Boycott,2001,PG,24 Feb 2001,118 min,Drama,Clark Johnson,"Stewart Burns (book), Herman Daniel Farrell (t...","Jeffrey Wright, Terrence Howard, CCH Pounder, ...",Black Americans boycott the public buses durin...,...,movie,07 Dec 2017,N/A,"HBO Films, Shelby Stone Productions, Norman Tw...",N/A,True,NaN,NaN,NaN,NaN
3,Something's Gotta Give,2003,PG-13,12 Dec 2003,128 min,"Comedy, Drama, Romance",Nancy Meyers,Nancy Meyers,"Jack Nicholson, Diane Keaton, Keanu Reeves, Fr...",Harry Sanborn is an aged music industry exec w...,...,movie,16 Apr 2012,"$124,728,738","Warner Brothers, Waverly Films, Columbia Pictu...",N/A,True,NaN,NaN,NaN,NaN
4,Daydream Obsession,2003,N/A,23 Nov 2003,87 min,Romance,Thomas R. Smyth,Scott Yarbrough,"Joe Dain, Kris Black, Julian Coca, Danny Dallas",Clayton has a problem with his roommate Brian....,...,movie,N/A,N/A,N/A,N/A,True,NaN,NaN,NaN,NaN
5,Searching for Paradise,2002,R,24 Jun 2002,88 min,Drama,Myra Paci,Myra Paci,"Susan May Pratt, Chris Noth, Jeremy Davies, Jo...","A young woman, coping with the death of her fa...",...,movie,22 Mar 2005,N/A,Centre Street Productions,N/A,True,NaN,NaN,NaN,NaN
6,The Love Letter,1999,PG-13,21 May 1999,88 min,"Comedy, Drama, Romance",Peter Ho-Sun Chan,"Cathleen Schine (novel), Maria Maggenti (scree...","Kate Capshaw, Blythe Danner, Ellen DeGeneres, ...",The power of words and images to open hearts. ...,...,movie,10 Aug 2016,"$8,302,478",DreamWorks SKG,N/A,True,NaN,NaN,NaN,NaN
7,Rudolph the Red-Nosed Reindeer,1964,TV-G,06 Dec 1964,47 min,"Animation, Adventure, Comedy, Family, Fantasy,...",Larry Roemer,"Romeo Muller, Robert May (story), Johnny Marks...","Burl Ives, Larry D. Mann, Billie Mae Richards,...",Sam the snowman tells us the story of a young ...,...,movie,N/A,N/A,N/A,N/A,True,NaN,NaN,NaN,NaN
8,The Bad and the Beautiful,1952,Passed,09 Feb 1953,118 min,"Drama, Romance",Vincente Minnelli,"Charles Schnee (screenplay), George Bradshaw (...","Lana Turner, Kirk Douglas, Walter Pidgeon, Dic...","Told in flashback form, the film traces the ri...",...,movie,16 Apr 2012,N/A,"Metro Goldwyn Mayer, Loew&#39;s Inc.",N/A,True,NaN,NaN,NaN,NaN
9,Justice League,2001–2004,TV-PG,17 Nov 2001,22 min,"Animation, Action, Adventure, Drama, Family, F...",N/A,"Bob Kane, William Moulton Marston, Joe Shuster...","Carl Lumbly, Phil LaMarr, Michael Rosenbaum, G...","Forces of evil, chaos, and destruction await. ...",...,series,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,2


In [5]:
omdb.join_omdb(omdb_net_path, save=True)

,imdbID,Title,Year,imdbRating,Metascore,Genre,Plot
0,tt0579513,Under Pressure,1997,10.0,NaN,Comedy,Everyone contributes to Gwen's moneymaking sch...
1,tt0528465,Unforgiven,1992,9.8,NaN,Comedy,NaN
2,tt0574008,A Married Man,1984,9.4,NaN,Comedy,Martin and Ann throw a dinner party and invite...
3,tt0185906,Band of Brothers,2001,9.4,NaN,"Action, Drama, History, War","This is the story of ""E"" Easy Company, 506th R..."
4,tt1736672,Widespread Panic: The Earth Will Swallow You,2002,9.3,NaN,"Music, War",Documentary centered around Athens GA rock & r...
...,...,...,...,...,...,...,...
9616,tt0675506,Hulk,2003,NaN,NaN,NaN,NaN
9617,tt1266298,A River Runs Through It,1992,NaN,NaN,Documentary,NaN
9618,tt0438362,Redemption,2004,NaN,NaN,"Crime, Short",NaN
9619,tt0833643,Redemption,2004,NaN,NaN,NaN,An 82-year-old woman appears to have committed...


In [4]:
netflix = NetCleaner()

In [5]:
netflix.get_title_ratings(net_raw_path, data_path, save=True)

,netflix_id,netflix_rating,number_of_votes,Title
0,1,7.499086,547,Dinosaur Planet
1,2,7.117241,145,Isle of Man TT 2004 Review
2,3,7.282306,2012,Character
3,4,5.478873,142,Paula Abdul's Get Up & Dance
4,5,7.838596,1140,The Rise and Fall of ECW
...,...,...,...,...
17765,17766,6.386777,605,Where the Wild Things Are and Other Maurice Se...
17766,17767,7.343137,204,Fidel Castro: American Experience
17767,17768,5.678414,1362,Epoch
17768,17769,4.997185,6749,The Company


In [6]:
netflix.get_base_dataframe(data_path, omdb_net_path, save=True)

,netflix_id,netflix_rating,number_of_votes,Title,imdbID,Year,imdbRating,Metascore,Genre,Plot
0,1,7.499086,547,Dinosaur Planet,tt0389605,2003–,7.7,NaN,"Documentary, Animation, Family",A four-episode animated series charting the ad...
1,12,6.835165,546,My Favorite Brunette,tt0039645,1947,6.8,NaN,"Comedy, Crime, Mystery, Romance, Thriller","Baby photographer Ronnie Jackson, on death row..."
2,15,6.572414,290,Neil Diamond: Greatest Hits Live,tt0168093,1988,8.0,NaN,"Documentary, Music",NaN
3,17,5.806415,7108,7 Seconds,tt0417395,2005,4.8,NaN,"Action, Comedy, Crime, Thriller",When an experienced thief accidentally makes o...
4,18,7.568737,10722,Immortal Beloved,tt0110116,1994,7.4,NaN,"Biography, Drama, Music, Romance",The life and death of the legendary Ludwig van...
...,...,...,...,...,...,...,...,...,...,...
10023,17762,7.291498,32110,Gattaca,tt0119177,1997,7.8,64.0,"Drama, Sci-Fi, Thriller","In the not-too-distant future, a less-than-per..."
10024,17763,6.823710,1957,Interiors,tt0077742,1978,7.4,67.0,Drama,The story of a very dysfunctional family and w...
10025,17764,7.734224,64957,Shakespeare in Love,tt0138097,1998,7.1,87.0,"Comedy, Drama, History, Romance",Will Shakespeare is a known but struggling poe...
10026,17769,4.997185,6749,The Company,tt0335013,2003,6.3,73.0,"Drama, Music, Romance",An inside look at the world of ballet. With th...
